In [16]:
import urllib2
import json
import time
from bs4 import BeautifulSoup
import urllib
import re

In [17]:

# A few functions that we will make use of
def get_url(category, page_atr, state,order):
    """Returns URLs to scrape based on page_str parameter and state
       State can be 'live' or 'successful' and reflects a projects status in kickstarter.com
    """
    url = "https://www.kickstarter.com/discover/advanced?&category_id=" + str(category) + \
    "&woe_id=0&sort="+order+"&format=json"
    state = "&state=" + state
    page = "&page="
    return([url + state + page + str(i+1) for i in range(page_atr + 1)])


def get_project_profile(project):
    """ Takes HTML DOM structure of each project and extracts 
        meta inforation about the project
    """
    #initate empty dictionary to store scraped data
    project_profile = dict()
    
    project_profile["author"] = project["creator"]["name"]
    project_profile["currency"] = project["currency"]
    project_profile["deadline"] = project["deadline"]
    project_profile["id"] = project["id"]
    project_profile["category"] = {k:project["category"][k] for k in ["id", "slug"]}
    project_profile["goal"] = project["goal"]
    project_profile["pledged"] = project["pledged"]
    project_profile["state"] = project["state"]
    project_profile["launched_at"] = project["launched_at"]
    
    project_profile["blurb"] = project["blurb"]
    project_profile["name"] = project["name"]
    project_profile["backers_count"] = project["backers_count"]
    project_profile["spotlight"] = project["spotlight"]
    project_profile["url"] = project["urls"]["web"]["project"].replace("?ref=discovery", "/description")
    project_profile["scrape_date"] = int(time.time())
    
    try:
        project_profile["location"] = {k:project["location"][k] for k in ["name", "country", "state"]}
    except:
        project_profile["location"] = None
    return(project_profile)


def get_json(urls):
    """Takes a set of urls as an input and returns contents of the webpage in json format
    """
    import urllib2
    import json
    p_json_list = []
    for idx,url in enumerate(urls):
        try:
            if idx % 50 == 0 :
                print "trying" + str(idx)
            p_json_list.append(json.loads(urllib2.urlopen(url).read()))
        except:
            print("page does not exist" + str(idx + 1))
    
    return(p_json_list)



In [18]:

## dd/mm/yyyy format
date=time.strftime("%Y%m%d")


In [19]:
page_atr = 200

#Repeating the same process for live projects 
#call function to generate URLs
urls = get_url(0,page_atr, 'live', 'end_date')
#call function to collect json format information for projects that are successful
project_profile_list = get_json(urls)
#Extract information we want about projects from scraped project urls
project_list1 = [get_project_profile(p) for pjson in project_profile_list for p in pjson["projects"]]


project_list1[100]


urls = get_url(0,page_atr, 'live', 'newest')
#call function to collect json format information for projects that are successful
project_profile_list = get_json(urls)
#Extract information we want about projects from scraped project urls
project_list2 = [get_project_profile(p) for pjson in project_profile_list for p in pjson["projects"]]

project_list2[100]

trying0
trying50
trying100
trying150
trying200
page does not exist201
trying0
trying50
trying100
trying150
trying200
page does not exist201


{'author': u'Virginia Woods-Jack & Tannya Harricks',
 'backers_count': 1,
 'blurb': u'A joint artist residency to explore and record one of the most unique environments on earth. Support us in return for artworks!',
 'category': {'id': 1, 'slug': u'art'},
 'currency': u'AUD',
 'deadline': 1435987720,
 'goal': 3900.0,
 'id': 1119211262,
 'launched_at': 1433568520,
 'location': {'country': u'AU', 'name': u'Hervey Bay', 'state': u'QLD'},
 'name': u'Artist Residency on World Heritage Listed Fraser Island',
 'pledged': 25.0,
 'scrape_date': 1433663609,
 'spotlight': False,
 'state': u'live',
 'url': u'https://www.kickstarter.com/projects/1400824884/artist-residency-on-world-heritage-listed-fraser-i/description'}

In [20]:
project_list= project_list1 + project_list2

In [21]:
pids = set([item["id"] for item in project_list])

In [22]:
unique_projects = []
for pid in pids:
    for proj in project_list:
        if proj["id"] == pid:
            unique_projects.append(proj)
            break

In [23]:
#Save data to file for further use
project_meta_json = json.dumps(obj = unique_projects,indent = 4)
fd = open("../data/project_live_"+ date +"_meta.json", "w")
fd.write(project_meta_json)
fd.close()


In [24]:
def get_one_project_det(project_det, soup):
    
    #the reward values and rewards
    b=list(soup.find_all(class_="NS_projects__rewards_list js-project-rewards"))
    reward_value_list = [reward.string.split()[0] for reward in b[0]("h5",class_="mb1")]
    backer_list = [backers.string.split()[0]+' backers' for backers in b[0]("span", class_="num-backers mr1")]
    reward_item_list = [item("p")[0].get_text() for item in b[0]("div",class_="desc h5 mb2 break-word")]
    
    #removing currency sign from reward value
    reg = re.compile("\d.*",flags=re.U)
    reward_value_list = [reg.findall(reward.string.split()[0])[0] for reward in b[0]("h5",class_="mb1")]

    
    
    #risk information
    c=list(soup("div", class_="mb6 mb2"))
    risk=[item("p") for item in c][0]
    
    
    project_det = dict()
    project_det["title"] = soup.find_all("meta", property="og:title")[0].get("content")
    #project_det["blurb1"] = soup.find_all(property=re.compile("og:description"))[0].get("content") 
    project_det["creator_url"] = soup.find_all(property=re.compile("kickstarter:creator"))[0].get("content")
    project_det["no_backers"] = list(soup("div", class_="NS_projects__spotlight_stats")[0])[1].string
    project_det["goal"] = reg.findall(list(soup.find_all("span", class_="money " +currency.lower()+ " no-code")[2])[0].string)
    project_det["no_dollars_raised"] = list(soup("div", class_="NS_projects__spotlight_stats")[0])[3].string
    project_det["reward_backer_tup"] = (zip(reward_value_list, backer_list,reward_item_list))
    project_det["risk"] = [para.get_text() for para in risk]
    project_det["id"] = pid
    project_det["url"] = url
    project_det["description"]=soup("div", class_="full-description js-full-description responsive-media formatted-lists")[0].find_all("p")
    x=[item.get("data-duration")  for item in soup.find_all("span") ]
    project_det["campaign_duration"]=[item for item in x if item is not None][0]
    y=[item.get("data-hours-remaining")  for item in soup.find_all("span") ]
    project_det["campaign_hours_remaining"]=[item for item in y if item is not None][0]
   
    return(project_det)

In [25]:
#open url information
fd = open("../data/project_live_"+ date +"_meta.json", "r")
text = fd.read()
fd.close()

pmeta = json.loads(text)





In [26]:
pmeta[1]

{u'author': u'The Major Minus',
 u'backers_count': 37,
 u'blurb': u'Join the major minus as we create history with our first EP and music video!',
 u'category': {u'id': 14, u'slug': u'music'},
 u'currency': u'USD',
 u'deadline': 1434324968,
 u'goal': 5000.0,
 u'id': 876118028,
 u'launched_at': 1431732968,
 u'location': {u'country': u'US', u'name': u'San Diego', u'state': u'CA'},
 u'name': u'The Major Minus',
 u'pledged': 2185.0,
 u'scrape_date': 1433663291,
 u'spotlight': False,
 u'state': u'live',
 u'url': u'https://www.kickstarter.com/projects/1758690409/the-major-minus/description'}

In [27]:


def get_soup(url, pcounter):
    #Creating beautiful soup object from URL
    try:
        r = urllib.urlopen(url)
        return BeautifulSoup(r)
    
    except:
        print "Failing for count", url
            
    
    
    
def get_one_project_det(project_det, soup):
    
    #the reward values and rewards
    b=list(soup.find_all(class_="NS_projects__rewards_list js-project-rewards"))
    reward_value_list = [reward.string.split()[0] for reward in b[0]("h5",class_="mb1")]
    backer_list = [backers.string.split()[0]+' backers' for backers in b[0]("span", class_="num-backers mr1")]
    reward_item_list = [item("p")[0].get_text() for item in b[0]("div",class_="desc h5 mb2 break-word")]
    
    #removing currency sign from reward value
    reg = re.compile("\d.*",flags=re.U)
    reward_value_list = [reg.findall(reward.string.split()[0])[0] for reward in b[0]("h5",class_="mb1")]

    
    
    #risk information
    c=list(soup("div", class_="mb6 mb2"))
    risk=[item("p") for item in c][0]
    
    
    project_det = dict()
    project_det["title"] = soup.find_all("meta", property="og:title")[0].get("content")
    #project_det["blurb1"] = soup.find_all(property=re.compile("og:description"))[0].get("content") 
    project_det["creator_url"] = soup.find_all(property=re.compile("kickstarter:creator"))[0].get("content")
    project_det["no_backers"] = list(soup("div", class_="NS_projects__spotlight_stats")[0])[1].string
    project_det["goal"] = reg.findall(list(soup.find_all("span", class_="money " +currency.lower()+ " no-code")[2])[0].string)
    project_det["no_dollars_raised"] = list(soup("div", class_="NS_projects__spotlight_stats")[0])[3].string
    project_det["reward_backer_tup"] = (zip(reward_value_list, backer_list,reward_item_list))
    project_det["risk"] = [para.get_text() for para in risk]
    project_det["id"] = pid
    project_det["url"] = url
    project_det["description"]=soup("div", class_="full-description js-full-description responsive-media formatted-lists")[0].find_all("p")
    x=[item.get("data-duration")  for item in soup.find_all("span") ]
    project_det["campaign_duration"]=[item for item in x if item is not None][0]
    y=[item.get("data-hours-remaining")  for item in soup.find_all("span") ]
    project_det["campaign_hours_remaining"]=[item for item in y if item is not None][0]
   
    return(project_det)    
        

In [28]:
import time
import datetime
#call function created above to grab project details and loop through project for every url in pmeta file
# for homework purposes, limited this to 500 urls, but we'll pull more for final project
project_live_soup = dict()

for pcounter, project_header in enumerate(pmeta):
    try:
        if pcounter % 400 == 0  :
            print("trying project ", pcounter)
        if datetime.datetime.fromtimestamp(int(project_header["deadline"])).strftime('%Y%m%d')==date:   
            project_live_soup[project_header["id"]] = get_soup(project_header["url"],pcounter) 
    except:
        print("failed for project element", pcounter) 
        
print(len(project_live_soup))

('trying project ', 0)
('trying project ', 400)
('trying project ', 800)
('trying project ', 1200)
('trying project ', 1600)
('trying project ', 2000)
('trying project ', 2400)
('trying project ', 2800)
('trying project ', 3200)
('trying project ', 3600)
('trying project ', 4000)
('trying project ', 4400)
('trying project ', 4800)
('trying project ', 5200)
('trying project ', 5600)
('trying project ', 6000)
('trying project ', 6400)
106


In [29]:
len(project_live_soup.values())

106

In [30]:
import requests

for key, val in project_live_soup.items():
    file_name = "../data/livesoup/" + str(key) + ".html"
    with open(file_name,"w") as f:
        f.write(str(key) + "\n" + str(val))
    with open("../data/livesoup/filenames"+date+".txt", "a") as s:
        s.write(file_name)
        s.write("\n")

In [102]:
with open("../data/test.html") as  f:
    soup=BeautifulSoup(f)
    
soup    

<html><body><p>(806713350, <!DOCTYPE html>

</p><script>
  //<![CDATA[
    docElement = this.document.documentElement;
    docElement.className=docElement.className.replace(/\bno-js\b/,'') + 'js';
  //]]>
</script>
<title>Last of the Northern White Rhino by Chad Copeland — Kickstarter</title><script>
  //<![CDATA[
    window.report_errors = true;
    window.honeybadger_configuration = {
      api_key : "02ab109335c0dd1a93506d92f22217a7",
      environment : "production",
      component : "projects",
      action : "description",
      onerror : false
    };
  //]]>
</script>
<!--[if lt IE 9]>
  <script src="//html5shim.googlecode.com/svn/trunk/html5.js"></script>
<![endif]-->
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<script type="text/javascript">window.NREUM||(NREUM={});NREUM.info={"beacon":"bam.nr-data.net","errorBeacon":"bam.nr-data.net","licenseKey":"b04b883ad2","applicationID":"11086","transactionName":"cFteRUULX1wHFhtIEFxeVVJDFxxUBxdXSgtDQFleWQ==","qu

In [80]:

project_details = []
for key, value in project_live_soup.items()[0:1]:
    #try:
        project_details.append(get_one_project_det(key, value))
    #except:
        #print "This id " + str(key) + " fucked you up!!!"
  

AttributeError: 'NoneType' object has no attribute 'split'

In [84]:
get_one_project_det(project_live_soup.keys()[1],project_live_soup.values()[1])

IndexError: list index out of range